In [5]:
import requests
import json
import csv
import pandas as pd

In [52]:
# URL листинга автомобилей
URL = 'https://auto.ru/-/ajax/desktop/listing/'

# Заголовок
HEADER = {
    'Host': 'auto.ru',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18362',
    'Accept': '*/*',
    'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Referer': 'https://auto.ru/moskva/cars/bmw/used/?output_type=list&page=1',
    'x-client-app-version': '201911.26.155818',
    'x-csrf-token': 'a5446446051b69f4b14cb18507b6ebb9954c6cf76b93027f',
    'x-requested-with': 'fetch',
    'content-type': 'application/json',
    'Origin': 'https://auto.ru',
    'Content-Length': '157',
    'DNT': '1',
    'Connection': 'keep-alive',
    'Cookie': '_csrf_token=a5446446051b69f4b14cb18507b6ebb9954c6cf76b93027f; autoru_sid=a%3Ag5dde33da257four8ct4hhlt0a17h00o.5bc9aa797010bf795c3fae4e20dd2eef%7C1574843354139.604800.PHszmA0eR_XZhCgieItl5g.WeKyrJqm5x5NmWaOBK-TLSEyI0wo2Z27miqpLLFwee8; autoruuid=g5dde33da257four8ct4hhlt0a17h00o.5bc9aa797010bf795c3fae4e20dd2eef; suid=d6c67bcdfb90c719a1008ae312af2fc9.f9f984e280d08277d997ce0fac3c604c; from_lifetime=1574843440313; from=google-search; counter_ga_all7=1; X-Vertis-DC=sas; navigation_promo_seen-recalls=true',
    'Pragma': 'no-cache',
    'Cache-Control': 'no-cache'
}

# Параметры запроса: автомобили марки BMW с пробегом в Москве 
PARAMS = {
    "category": "cars",
    "section": "used",
    "output_type": "list",
    "page": 1,
    "catalog_filter": [{"mark":"BMW"}],
    "geo_radius": 200,
    "geo_id": [213]
}

In [54]:
# Список для хранения авто
offers = []

for i in range(1, 200):
    PARAMS['page'] = i
    
    # запрашиваем каждую страницу и берем из ответа список авто
    response = requests.post(url=URL, json=PARAMS, headers=HEADER)
    offers.extend(response.json()['offers'])
    
# записываем в файл чтобы не обращаться каждый раз    
with open('offers.json', 'w') as f:
    json.dump(offers, f)

# Количество авто по запросам
print(len(offers))

7478


In [80]:
data_columns = (
    'id', 'brand', 'color', 'owners_number', 'pts', 'year', 'vin_resolution', 'tax_by_year', 'mileage',
    'not_beaten', 'body_type', 'doors', 'class', 'trunk_v_min', 'trunk_v_max', 'equipment', 'engine_volume',
    'engine_type', 'gear_type', 'transmission', 'power', 'power_kvt', 'acceleration', 'fuel_rate', 'osago',
    'steering_wheel', 'warranty', 'purchase_year', 'purchase_month', 'warranty_expire_year', 'warranty_expire_month',
    'generation', 'price_segment', 'last_year_price_percentage_diff', 'vendor', 'clearance_min', 'price'
)

In [181]:
# Выполнить, если нужно заново загрузить список авто
with open('offers.json', 'r') as f:
    offers = json.load(f)

In [182]:
def prepare_data(offers):
    result = []
    
    for index in range(len(offers)):
        item = offers[index]
        new_element = dict.fromkeys(data_columns, None)
        
        new_element['id'] = index
        
        if item.get('vehicle_info').get('mark_info') is not None:
            new_element['brand'] = item.get('vehicle_info').get('mark_info').get('name')
            
        new_element['color'] = item.get('color_hex')
        new_element['owners_number'] = item.get('documents').get('owners_number')
        new_element['pts'] = item.get('documents').get('pts')
        new_element['year'] = item.get('documents').get('year')
        new_element['vin_resolution'] = item.get('documents').get('vin_resolution')
        
        if item.get('owner_expenses').get('transport_tax') is not None:
            new_element['tax_by_year'] = item.get('owner_expenses').get('transport_tax').get('tax_by_year')
            
        new_element['mileage'] = item.get('state').get('mileage')
        new_element['not_beaten'] = item.get('state').get('state_not_beaten')
        new_element['body_type'] = item.get('vehicle_info').get('configuration').get('body_type')
        new_element['doors'] = item.get('vehicle_info').get('configuration').get('doors_count')
        new_element['class'] = item.get('vehicle_info').get('configuration').get('auto_class')
        new_element['trunk_v_min'] = item.get('vehicle_info').get('configuration').get('trunk_volume_min')
        new_element['trunk_v_max'] = item.get('vehicle_info').get('configuration').get('trunk_volume_max')
        new_element['equipment'] = list(item.get('vehicle_info').get('equipment').keys())
        new_element['engine_volume'] = item.get('vehicle_info').get('tech_param').get('displacement')
        new_element['engine_type'] = item.get('vehicle_info').get('tech_param').get('engine_type')
        new_element['gear_type'] = item.get('vehicle_info').get('tech_param').get('gear_type')
        new_element['transmission'] = item.get('vehicle_info').get('tech_param').get('transmission')
        new_element['power'] = item.get('vehicle_info').get('tech_param').get('power')
        new_element['power_kvt'] = item.get('vehicle_info').get('tech_param').get('power_kvt')
        new_element['acceleration'] = item.get('vehicle_info').get('tech_param').get('acceleration')
        new_element['fuel_rate'] = item.get('vehicle_info').get('tech_param').get('fuel_rate')
        
        if item.get('owner_expenses').get('osago_insurance') is not None:
            new_element['osago'] = item.get('owner_expenses').get('osago_insurance')[0].get('price')
            
        new_element['steering_wheel'] = item.get('vehicle_info').get('steering_wheel')
        new_element['warranty'] = item.get('documents').get('warranty')
        
        if item.get('documents').get('purchase_date') is not None:
            new_element['purchase_year'] = item.get('documents').get('purchase_date').get('year')
            new_element['purchase_month'] = item.get('documents').get('purchase_date').get('month')
            
        if item.get('documents').get('warranty_expire') is not None:
            new_element['warranty_expire_year'] = item.get('documents').get('warranty_expire').get('year')
            new_element['warranty_expire_month'] = item.get('documents').get('warranty_expire').get('month')
            
        if item.get('vehicle_info').get('super_gen') is not None:
            new_element['generation'] = item.get('vehicle_info').get('super_gen').get('ru_name')
            new_element['price_segment'] = item.get('vehicle_info').get('super_gen').get('price_segment')
        
        if item.get('additional_info').get('price_stats') is not None:
            new_element['last_year_price_percentage_diff'] = item.get('additional_info').get('price_stats').get('last_year_price_percentage_diff')
            
        new_element['vendor'] = item.get('vehicle_info').get('vendor')
        new_element['clearance_min'] = item.get('vehicle_info').get('tech_param').get('clearance_min')
        new_element['price'] = item.get('price_info').get('price')
        
        result.append(new_element)
        
    return result

In [183]:
offers = prepare_data(offers)
offers[0]

{'id': 0,
 'brand': 'BMW',
 'color': '040001',
 'owners_number': 3,
 'pts': 'ORIGINAL',
 'year': 2012,
 'vin_resolution': 'OK',
 'tax_by_year': 45900,
 'mileage': 170000,
 'not_beaten': True,
 'body_type': 'ALLROAD_5_DOORS',
 'doors': 5,
 'class': 'J',
 'trunk_v_min': 570,
 'trunk_v_max': 1450,
 'equipment': ['wheel-heat', 'front-seats-heat-vent'],
 'engine_volume': 2979,
 'engine_type': 'GASOLINE',
 'gear_type': 'ALL_WHEEL_DRIVE',
 'transmission': 'AUTOMATIC',
 'power': 306,
 'power_kvt': 225,
 'acceleration': 6.7,
 'fuel_rate': 10.1,
 'osago': 4375,
 'steering_wheel': 'LEFT',
 'warranty': None,
 'purchase_year': 2018,
 'purchase_month': 6,
 'warranty_expire_year': None,
 'warranty_expire_month': None,
 'generation': '1 (E71) Рестайлинг',
 'price_segment': 'PREMIUM',
 'last_year_price_percentage_diff': -12,
 'vendor': 'EUROPEAN',
 'clearance_min': 212,
 'price': 1449999}

In [184]:
data = pd.DataFrame(offers)
data.head(10)

,id,brand,color,owners_number,pts,year,vin_resolution,tax_by_year,mileage,not_beaten,...,purchase_year,purchase_month,warranty_expire_year,warranty_expire_month,generation,price_segment,last_year_price_percentage_diff,vendor,clearance_min,price
0,0,BMW,040001,3,ORIGINAL,2012,OK,45900,170000,True,...,2018.0,6.0,NaN,NaN,1 (E71) Рестайлинг,PREMIUM,-12.0,EUROPEAN,212.0,1449999.0
1,1,BMW,040001,1,ORIGINAL,2016,OK,57150,44500,True,...,2017.0,3.0,NaN,NaN,3 (F15),PREMIUM,-11.0,EUROPEAN,209.0,4350000.0
2,2,BMW,040001,3,ORIGINAL,2018,OK,9310,90000,True,...,2018.0,10.0,2021.0,11.0,7 (G30/G31),PREMIUM,-15.0,EUROPEAN,139.0,1900000.0
3,3,BMW,EE1D19,3,ORIGINAL,2009,OK,2875,193951,True,...,NaN,NaN,NaN,NaN,None,PREMIUM,-8.0,EUROPEAN,145.0,489000.0
4,4,BMW,4A2197,3,ORIGINAL,1998,OK,17625,178350,True,...,NaN,NaN,NaN,NaN,4 (E39),PREMIUM,-13.0,EUROPEAN,120.0,225000.0
5,5,BMW,CACECB,3,DUPLICATE,1996,OK,2875,240000,True,...,NaN,NaN,NaN,NaN,3 (E36),PREMIUM,-14.0,EUROPEAN,110.0,175000.0
6,6,BMW,4A2197,3,ORIGINAL,1995,OK,9650,275000,True,...,NaN,NaN,NaN,NaN,3 (E36),PREMIUM,-14.0,EUROPEAN,110.0,250000.0
7,7,BMW,040001,2,ORIGINAL,2019,OK,187500,12923,True,...,NaN,NaN,NaN,NaN,6 (F90),PREMIUM,-18.0,EUROPEAN,128.0,6997000.0
8,8,BMW,CACECB,3,ORIGINAL,1999,OK,5250,250000,True,...,NaN,NaN,NaN,NaN,4 (E39),PREMIUM,-13.0,EUROPEAN,120.0,215000.0
9,9,BMW,CACECB,3,ORIGINAL,2002,OK,9600,192000,True,...,2017.0,8.0,NaN,NaN,4 (E46),PREMIUM,-14.0,EUROPEAN,130.0,390000.0


In [185]:
data.isnull().sum()

id                                    0
brand                                 0
color                                 0
owners_number                         0
pts                                   0
year                                  0
vin_resolution                       15
tax_by_year                           0
mileage                               0
not_beaten                            0
body_type                             0
doors                                 0
class                                 0
trunk_v_min                           8
trunk_v_max                        3511
equipment                             0
engine_volume                         0
engine_type                           0
gear_type                             0
transmission                          0
power                                 0
power_kvt                             0
acceleration                        212
fuel_rate                           490
osago                               114
